# 컴퓨터 비전 데이터셋 2
**COCO (Common Object in Context)** 는 컴퓨터 비전 연구에서 모델의 학습 및 성능 평가 등 다양한 분야에서 많이 사용하는 데이터셋

이 데이터를 로컬로 다운받기에는 용량이 크고 다운로드 받기 위해서는 많은 시간 필요

**COCO API**를 이용하면 Annotation(어노테이션) 자료만 있으면 인터넷에서 바로 이미지를 확인 가능

## 1. 준비하기
**COCO API**를 사용하기 위해서는 아래의 라이브러리가 필요

In [ ]:
%matplotlib inline
from pycocotools.coco import COCO

import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

`val2017` 폴더에는 Annotation 정보가 저장된 JSON 파일이 준비되어 있음

In [ ]:
# 현재 디렉토리를 가리키는 변수 dataDir을 설정합니다.
dataDir='.'  

# 사용할 데이터 세트의 타입을 'val2017'로 설정하는 변수 dataType을 정의합니다.
dataType='val2017'  

# COCO 데이터 세트의 어노테이션 파일 경로를 문자열 포맷팅을 사용하여 구성하고, 이 경로를 변수 annFile에 저장합니다. 
# 여기서 '{}'는 format 메서드에 의해 dataDir과 dataType 변수의 값으로 대체됩니다.
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)  

print(str(annFile))

In [ ]:
# COCO 클래스의 인스턴스를 생성하고, 이전에 정의된 어노테이션 파일 경로(annFile)를 사용하여 초기화합니다. 
# 이 인스턴스를 변수 coco에 할당하여, COCO 데이터 세트에 대한 다양한 작업(예: 어노테이션 로딩, 이미지 정보 조회 등)을 수행할 수 있습니다.
coco=COCO(annFile)  

COCO 데이터셋의 모든 카테고리와 상위 카테고리 정보 확인

자율주행 자동차를 위해서는 자동차, 오토바이 등 도로 위의 객체도 중요하지만 돌발 상황에 따른 사람, 전봇대, 유모차 등 다양한 카테고리가 필요할 수 있기 때문

In [ ]:
# COCO 데이터 세트에서 모든 카테고리의 정보를 로드합니다. 
# coco.getCatIds()는 모든 카테고리의 ID를 가져오고, loadCats 메서드는 이 ID들에 해당하는 카테고리 정보를 반환합니다.
cats = coco.loadCats(coco.getCatIds())  

# 각 카테고리의 이름을 리스트 컴프리헨션을 사용하여 리스트 nms에 저장합니다.
nms=[cat['name'] for cat in cats]  

# 카테고리 이름을 공백으로 구분하여 한 줄에 출력합니다.
print('COCO categories: \n{}\n'.format(' '.join(nms))) 


In [ ]:
# 각 카테고리의 상위 카테고리를 리스트 컴프리헨션을 사용하여 추출하고, 이를 set()을 사용하여 중복을 제거한 후 nms에 저장합니다.
nms = set([cat['supercategory'] for cat in cats])  

# 상위 카테고리를 공백으로 구분하여 한 줄에 출력합니다.
print('COCO supercategories: \n{}'.format(' '.join(nms)))  

## 2. 이미지 출력하기
원하는 카테고리를 입력하면 주어진 카테고리를 포함하는 모든 이미지를 가져오고 그 중에서 하나의 이미지를 출력하는 실습 진행

`getCatIds` 함수에 원하는 카테고리를 입력하면 카테고리 ID를 반환

In [ ]:
# 'car', 'motorcycle', 'bus', 'truck' 카테고리에 해당하는 카테고리 ID들을 가져와 catIds에 할당합니다.
catIds = coco.getCatIds(catNms=['car','motorcycle','bus', 'truck'])
catIds

`getImgIds` 함수는 카테고리 아이디를 넣으면 이미지의 ID를 반환


In [ ]:
# 위에서 얻은 카테고리 ID에 해당하는 모든 이미지의 ID를 가져와 imgIds에 할당합니다.
imgIds = coco.getImgIds(catIds=catIds)
imgIds

`loadImgs`함수를 이용하면 이미지ID에 해당하는 이미지 정보를 반환

In [ ]:
# imgIds 리스트에서 무작위로 하나의 이미지 ID를 선택하고, 해당 이미지 정보를 가져와 img에 할당합니다. 
# [0]은 loadImgs 메서드가 리스트 형태로 결과를 반환하기 때문에 첫 번째 이미지 정보만을 선택하기 위함입니다.
img = coco.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]  
img

In [ ]:
# 특정 이미지 ID(336232)를 가진 이미지의 ID를 다시 가져와 imgIds에 할당합니다. 
imgIds = coco.getImgIds(imgIds = [336232])  
imgIds

In [ ]:
img = coco.loadImgs(imgIds)[0]
img

img 변수에 있는 "coco_url" 주소를 통해 이미지를 불러옴 (인터넷 연결 필요)

불어온 이미지는 matplotlib 라이브러리를 이용하여 출력

In [ ]:
# 이미지를 로드하고 표시합니다.
# URL을 사용하여 이미지 로드
# 이미지의 COCO URL을 사용하여 인터넷에서 직접 이미지를 로드합니다. 이 방법은 이미지가 웹에 공개적으로 접근 가능할 때 유용합니다.
I = io.imread(img['coco_url'])  


# 이미지를 표시할 때 축을 끕니다.
plt.axis('off')  


# matplotlib의 imshow 함수를 사용하여 이미지 I를 표시하고 화면에 보여줍니다.
plt.imshow(I)  
plt.show()  


## 3. Annotation 표시하기

가져온 이미지에는 바운딩 박스의 위치 좌표, 세그맨테이션 정보를 가지고 있는 **Annotaiont JSON 파일**이 있음

해당 파일을 읽어옴으로써 이미지 위에 표시하는 실습을 진행


Annotation의 정보를 읽기 위해서는 `getAnnIds`와 `loadAnns` 함수가 필요

* `getAnnIds` : 특정 이미지 ID에 해당하는 모든 어노테이션 ID들을 조회하는 함수
* `loadAnns` :  조회한 어노테이션 ID에 해당하는 어노테이션 정보를 로드하는 함수

In [ ]:
# 해당 이미지의 모든 어노테이션 가져오기

# img['id']는 조회할 이미지의 ID, catIds는 특정 카테고리에 속하는 어노테이션만 조회하고 싶을 때 사용하는 카테고리 ID의 리스트입니다. 
annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)

# 조회한 어노테이션 ID에 해당하는 어노테이션 정보를 로드합니다.
anns = coco.loadAnns(annIds)
anns[0]

### 3.1 바운딩 박스(Bounding Box) 표시하기

In [ ]:
for ann in anns:
    category_id = ann['category_id']
    bbox = ann['bbox']

    category = ''
    
    if category_id == 3 :
        category = 'car'

    elif category_id == 4 :
        category = 'motorcycle'

    elif category_id == 6 :
        category = 'bus'

    elif category_id == 8 :
        category = 'truck'


    else : 
        category = 'else'


    print(category, " ", category_id, " ",bbox)

In [ ]:
# matplotlib를 사용하여 이미지 I를 표시하고, 축 정보는 표시하지 않습니다.
plt.imshow(I); plt.axis('off')  


# 어노테이션 리스트인 anns를 순회하며 각 어노테이션에 대해 반복합니다.
for ann in anns:  
    
    # 현재 어노테이션의 바운딩 박스 정보를 bbox 변수에 저장합니다. bbox는 [x, y, width, height] 형식의 리스트입니다.
    bbox = ann['bbox']  
    
    # 현재 축(plt.gca())에 바운딩 박스를 나타내는 사각형을 추가합니다. 
    # 사각형의 시작점은 (bbox[0], bbox[1])이며, 너비와 높이는 각각 bbox[2], bbox[3]입니다. 
    # 선의 두께(linewidth)는 2, 선의 색상(edgecolor)은 빨간색으로 설정하고, 면의 색상(facecolor)은 채우지 않습니다.
    plt.gca().add_patch(plt.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=2, edgecolor='red', facecolor='none'))  

# 시각화한 이미지와 바운딩 박스를 화면에 표시합니다.
plt.show()  


In [ ]:
plt.imshow(I); plt.axis('off')  

for ann in anns:
    category_id = ann['category_id']
    bbox = ann['bbox']

    category = ''
    
    if category_id == 3 :
        plt.gca().add_patch(plt.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=2, edgecolor='red', facecolor='none'))  


    elif category_id == 4 :
        plt.gca().add_patch(plt.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=2, edgecolor='blue', facecolor='none'))  


    elif category_id == 6 :
        plt.gca().add_patch(plt.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=2, edgecolor='Yellow', facecolor='none'))  


    elif category_id == 8 :
        plt.gca().add_patch(plt.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=2, edgecolor='Green', facecolor='none'))  


    else : 
        plt.gca().add_patch(plt.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=2, edgecolor='pink', facecolor='none'))  

plt.show()  

### 3.2 세그맨테이션(Segmentation) 표시하기

In [ ]:
# 이미지와 해당 인스턴스 어노테이션을 로드하고 표시합니다.

# 이미지 I를 표시하고 축을 끕니다.
plt.imshow(I); plt.axis('off')  


# 특정 이미지 ID와 카테고리 ID에 해당하는 어노테이션 ID들을 가져옵니다. 
# 여기서 iscrowd=None은 군중 객체(crowd objects)를 포함하거나 제외하지 않음을 의미합니다.
annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)  


# 위에서 얻은 어노테이션 ID들에 해당하는 어노테이션 정보를 로드합니다.
anns = coco.loadAnns(annIds)  


# 로드된 어노테이션 정보를 이미지 위에 표시합니다. 이 함수는 바운딩 박스 또는 세그멘테이션 마스크를 이미지 위에 그리는 등, 어노테이션 정보를 시각화합니다.
coco.showAnns(anns)  


## [부록] Caption Annotaion 확인하기

**Caption Annotation**은 이미지에 대한 설명적인 텍스트를 제공하는 주석

해당 파일을 통해 이미지의 내용, 발생하는 사건, 객체의 관계 및 활동 등을 설명할 수 있음

In [ ]:
# 캡션 어노테이션을 위한 COCO API를 초기화합니다.
annFile = '{}/annotations/captions_{}.json'.format(dataDir,dataType)  
coco_caps=COCO(annFile)  

In [ ]:
# 이미지에 대한 캡션 어노테이션을 로드하고 표시합니다.

# 특정 이미지 ID에 해당하는 캡션 어노테이션 ID들을 가져옵니다. 이를 위해 coco_caps 인스턴스의 getAnnIds 메서드를 사용합니다.
annIds = coco_caps.getAnnIds(imgIds=img['id']);  


# 위에서 얻은 어노테이션 ID들에 해당하는 캡션 어노테이션 정보를 로드합니다.
anns = coco_caps.loadAnns(annIds)  


# 로드된 캡션 어노테이션을 출력합니다. showAnns 메서드는 각 캡션 어노테이션을 순회하면서 텍스트 형태의 캡션을 표시합니다.
coco_caps.showAnns(anns)  


# 같은 이미지 I를 다시 표시하고, 축을 끕니다. 이 코드 라인은 이미지를 다시 표시하기 위해 사용되며, 이미지 위에 어노테이션을 그리는 대신 캡션 텍스트를 출력합니다.
plt.imshow(I)
plt.axis('off')
plt.show()  
